In [18]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),              # Resize 到 CNN 接收的大小
    transforms.RandomHorizontalFlip(),          # 数据增强：随机水平翻转
    transforms.ToTensor(),                      # 转为 Tensor
    transforms.Normalize(                       # 标准化
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset_path = '../datasets/trashnet'

# 自动读取子文件夹并赋予标签
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 查看类别索引（非常重要）
print("类别映射:", full_dataset.class_to_idx)

# 划分为训练集和验证集（比如 80% 训练 + 20% 验证）
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 构建 Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

类别映射: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [19]:
images, labels = next(iter(train_loader))
print("一个 batch 的图像形状：", images.shape)
print("对应的标签：", labels[:5])

一个 batch 的图像形状： torch.Size([32, 3, 224, 224])
对应的标签： tensor([4, 1, 4, 1, 0])


In [20]:
# 1. 加载模型 & 设置 device
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("使用设备：", device)

# 2. 加载模型结构并替换 fc 层（⚠️ 先改结构！）
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 6)  # 👈 先替换最后一层
model.load_state_dict(torch.load('../models/recycling_resnet18_15ep.pth', map_location=device))  # 👈 再加载参数
model = model.to(device)

# 3. 冻结特征提取层
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():  # 但 fc 层需要训练
    param.requires_grad = True

# 4. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 5. 继续训练
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Train Accuracy: {train_acc:.4f}")

# 6. 保存更新后的模型
torch.save(model.state_dict(), 'recycling_resnet18.pth')

使用设备： cpu
Epoch 1/5 - Loss: 28.0911 - Train Accuracy: 0.8367
Epoch 2/5 - Loss: 24.7472 - Train Accuracy: 0.8689
Epoch 3/5 - Loss: 24.5961 - Train Accuracy: 0.8723
Epoch 4/5 - Loss: 25.2018 - Train Accuracy: 0.8570
Epoch 5/5 - Loss: 26.0755 - Train Accuracy: 0.8580
